## 管理层信息抽取

In [1]:
! cp /Users/alchemy_taotaox/Desktop/next/my_test/* ../corpus/

In [2]:
! ls ../corpus

kg_expert.pdf     year_report.pdf   year_report_2.pdf


## 算法基本思路
1. 扫描pdf 获得页眉页脚
2. 扫描pdf 获得最小和最大的x坐标
    如果一个字符是最小的坐标，那么就和之前的行合并为一个段落
    如果一个字符是最大的坐标，那么就和之前的
2. 提取文档中的表格和字符串。调整

In [12]:
from pdfplumber.pdf import PDF
import re
from collections import Counter

def is_header(str, header):
    if header is None:
        return False
    return str == header

def is_foot(str, foot):
    if foot is None:
        return False
    
    s = re.sub(r'\d+', '#num#', str)
    return s == foot

def get_foot_header(pdf:PDF, check_pages_num=20, threshold = 0.9):
    head_counter = Counter()
    foot_counter = Counter()
    pages_num = len(pdf.pages)
    if pages_num < check_pages_num:
        check_pages_num = pages_num
    for i in range(check_pages_num):
        text_lines = pdf.pages[i].extract_words()
        top_line = text_lines[0]['text']
        bottom_line = text_lines[-1]['text']
        bottom_line = re.sub(r'\d+', '#num#', bottom_line)
        head_counter.update([top_line])
        foot_counter.update([bottom_line])
        
    most_common_head_word, count_head = head_counter.most_common(1)[0]
    most_common_foot_word, count_foot = foot_counter.most_common(1)[0]
    head, foot = None, None
    if count_head *1./check_pages_num >= threshold:
        head = most_common_head_word
    if count_foot *1./check_pages_num >= threshold:
        foot = most_common_foot_word
    return head, foot
    
    

In [8]:
import pdfplumber

filename = "../corpus/year_report_2.pdf"
pdf = pdfplumber.open(filename)
page = pdf.pages[10]
page = pdf.pages[15]
print(page.extract_text())

上海安诺其集团股份有限公司2022年年度报告全文
截止2022年12月31日，公司已
建成分散染料产能30,000吨，在
分散染料 82,700吨 80.14% 52,700吨
建产能52,700吨，具体可参见本
报告第三节“七、投资状况分析”
2021年7月，公司建成了活性染
料产能14,750吨并开始试生产，
活性染料 14,750吨 12.79% 0
具体可参见本报告第三节“七、投
资状况分析”。
截止2022年12月31日，公司已
建成染料中间体18,177吨，其中
2,000吨中间体2-溴-4-硝基-6-氯
中间体 88,000吨 50.96% 69,823吨
苯于2021年7月建设完成并开始
试生产。具体可参见本报告第三节
“七、投资状况分析”。
主要化工园区的产品种类情况
主要化工园区 产品种类
山东河口经济开发区 分散染料
蓬莱市北沟化工新材料产业聚集区 染料中间体
报告期内正在申请或者新增取得的环评批复情况
 适用 不适用
报告期内，30000吨染料中间体建设项目二期项目环评正在申请中。
报告期内上市公司出现非正常停产情形
□适用 不适用
相关批复、许可、资质及有效期的情况
适用 □不适用
所属公司 名称 证书编号 有效期 备注
2022年11月22日至
东营安诺其纺 排污许可证 91370500679213060H001V --
2027年11月21日
织材料有限公
2020年1月20日至
司 安全标准化证书 无 --
2023年1月20日
2021年09月27日至
排污许可证 913706840839718641001V --
2026年09月26日
2022年7月4日至
烟台安诺其精 安全生产许可证 （鲁）WH安许证字（2022）060223号 --
2024年4月8日
细化工有限公
2021年12月27日至
司 安全生产标准化证书 （鲁）AQBWH II 202100202 --
2024年12月
2022年8月20日至
危险化学品登记证 370610228 --
2025年8月19日
从事石油加工、石油贸易行业
□是 否
从事化肥行业
□是 否
从事农药行业
□是 否
从事氯碱、纯碱行业
16


In [15]:
from operator import itemgetter

def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

head, foot = get_foot_header(pdf)
print(head)
print(foot)
tables = page.find_tables()
table_bboxes = [i.bbox for i in tables]
tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
non_table_words = [word for word in page.extract_words() if not any(
    [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
lines = []
for cluster in pdfplumber.utils.cluster_objects(
        non_table_words + tables, itemgetter('top'), tolerance=5):
    if 'text' in cluster[0]:
        lines.append(' '.join([i['text'] for i in cluster if not is_foot(i['text'], foot) and not is_header(i['text'], head)]))
    elif 'table' in cluster[0]:
        lines.append(cluster[0]['table'])
# print(lines)

for item in lines:
    print(item)

上海安诺其集团股份有限公司2022年年度报告全文
#num#

[['分散染料', '82,700吨', '80.14%', '52,700吨', '截止2022年12月31日，公司已\n建成分散染料产能30,000吨，在\n建产能52,700吨，具体可参见本\n报告第三节“七、投资状况分析”'], ['活性染料', '14,750吨', '12.79%', '0', '2021年7月，公司建成了活性染\n料产能14,750吨并开始试生产，\n具体可参见本报告第三节“七、投\n资状况分析”。'], ['中间体', '88,000吨', '50.96%', '69,823吨', '截止2022年12月31日，公司已\n建成染料中间体18,177吨，其中\n2,000吨中间体2-溴-4-硝基-6-氯\n苯于2021年7月建设完成并开始\n试生产。具体可参见本报告第三节\n“七、投资状况分析”。']]
主要化工园区的产品种类情况
[['', '主要化工园区', '', '', '产品种类', ''], ['山东河口经济开发区', None, None, '分散染料', None, None], ['蓬莱市北沟化工新材料产业聚集区', None, None, '染料中间体', None, None]]
报告期内正在申请或者新增取得的环评批复情况
 适用 不适用
报告期内，30000吨染料中间体建设项目二期项目环评正在申请中。
报告期内上市公司出现非正常停产情形
□适用 不适用
相关批复、许可、资质及有效期的情况
适用 □不适用
[['所属公司', '名称', '证书编号', '有效期', '备注'], ['', '排污许可证', '91370500679213060H001V', '2022年11月22日至\n2027年11月21日', '--'], ['东营安诺其纺', None, None, None, None], ['织材料有限公', None, None, None, None], [None, '安全标准化证书', '无', '2020年1月20日至\n2023年1月20日', '--'], ['司', None, None, None, None], ['', None, None, None, None], ['', '排污许可证', '913706

In [14]:
head, foot = get_foot_header(pdf)
print(head)
print(foot)

上海安诺其集团股份有限公司2022年年度报告全文
#num#
